# Variational quantum eigensolver

$$\left| \psi \right\rangle $$


aaaaaaaaaaaaaaaaaaaaaaa

$$| \psi \rangle $$

aaaaaaaaaaaaaaaaaaaaaaa

$$| \psi> $$

aaaaaaaaaaaaaaaaaa

$$\small | 3x+7 \bigg \rangle$$

Variational quantum eigensolver (VQE) is a hybrid quantum-classical algorithm [1] that finds the smallest eigenvalue (and corresponding eigenvector) of a given Hamiltonian. One of the main applications of the algorithm is finding ground state energy of the molecules. It has a big advantage over [IPEA](https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/iterative_phase_estimation.ipynb) (iterative phase estimation algorithm) and PEA (phase estimation algorithm) algorithms, that also can be used for finding the ground state energy of a molecule. The main advantage is that VQE uses a much small circuit depth (or gates) then IPEA and PEA, what is very important for NISQ (Noisy Intermediate-Scale Quantum) era quantum computation. In the NISQ era (now!) we are working with qubits that are very noisy because they are not isolated from the environment well enough. Thus, there is small and finite time to work with qubits until they will be "spoiled", because of the environment, imperfect gates and etc. This restriction gives a big advantage to those algorithms (like VQE) that are using small depth circuits. 

The principle of the VQE algorithm is as follows. We have a Hamiltonian that can be expressed by the sum of tensor products of Pauli operators (Pauli terms):

$$H = 0.4 \cdot IX + 0.6 \cdot IZ + 0.8 \cdot XY$$

For a given $| \psi \rangle$ we want to measure the expectation value of the Hamiltonian:

$$<H> = \langle \psi | H | \psi \rangle = 0.4 \cdot \langle \psi | IX | \psi \rangle + 0.6 \cdot \langle \psi | IZ | \psi \rangle + 0.8 \cdot \langle \psi | XY | \psi \rangle$$

How one can see the $<H>$ expectation value could be computed by adding the expectation values of its parts (Pauli terms). The algorithm does exactly that. It constructs a quantum circuit for each Pauli term and computes the expectation value of the corresponding Pauli term. After it, the algorithm sums all calculated expectation values of Pauli terms and obtains the expectation value of the $H$. In this algorithm, we will do this routine of estimating the expectation value over and over again for different trial wavefunctions (ansatz) $| \psi \rangle$. 

It is known that the eigenvector $| \psi_g \rangle$ that minimizes the expectation value corresponds to the eigenvector of $H$ that has the smallest eigenvalue [1]. So, basically we can try all possible rial wavefunctions (ansatz) $| \psi \rangle$s to find the $| \psi_g \rangle$ that has the smallest expectation value (eigenvalue). Here the question is how we create those trial states? In the algorithm, the trial states are created from a parametrized circuit. By changing the parameters one obtains different wavefunctions (ansatz states) [1]. If your circuit with its parameters is good enough you will have access to more or less all possible ansatz states [2]. Otherwise, if the circuit will not have a possibility to generate our desired $| \psi_g \rangle$ it will be impossible to find the right solution.

The parameters of the state preparation circuit are controlled by a classical computer. At each step, the classical computer will change the parameters by using some optimization method in order to create an ansatz state that will have a smaller expectation value then previous ansatz states had. This way we will use a classical computer and quantum computer together to archive our goal (to find the ground state energy). That's way, VQE is a quantum-classical hybrid algorithm.

<img src="images/vqe_parts.png" alt="drawing" width="600"/>

This image is taken from the [1] paper. Here are the main parts of the VQE algorithm. The *Classical feedback decision* is some optimization method that changes the parameters of the *Quantum state preparation*. With different *Quantum modules* the algorithm calculates expectation values of each Pauli term and then it adds them by using *Classical adder* via classical computer. Then the algorithm returns to *Classical feedback decision* to choose better parameters for the *Quantum state preparation*. VQE repeats this procedure until the optimization method is satisfied with the obtained result. Note, that we will use these names in the image as method names in our code.

Now let's see how the algorithm works. Firstly, we should import all the packages that we will use.

In [1]:
import numpy as np
from random import random
from scipy import array
from scipy.optimize import minimize

from qiskit import *
from qiskit.extensions.standard import *
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import ExactEigensolver

For simplicity we will consider one qubit Hamiltonian consisting of a sum of 4 Pauli operators:

$$H = a \cdot I + b \cdot Z + c \cdot X + d \cdot Y$$

here are matrix representations of each Pauli operators:

$$
I = \begin{pmatrix}
1 & 0\\
0 & 1
\end{pmatrix}
\qquad
Z = \begin{pmatrix}
1 & 0\\
0 & -1
\end{pmatrix}
\qquad
X = \begin{pmatrix}
0 & 1\\
1 & 0
\end{pmatrix}
\qquad
Y = \begin{pmatrix}
0 & -i\\
i & 1
\end{pmatrix}
$$

The following method creates $H$ for a given $a$, $b$, $c$, $d$ coefficients.

In [2]:
def hamiltonian_operator(a, b, c, d):
    """
    Creates a*I + b*Z + c*X + d*Y pauli sum 
    that will be our Hamiltonian operator.
    
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "I"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "Z"},
                   {"coeff": {"imag": 0.0, "real": c}, "label": "X"},
                   {"coeff": {"imag": 0.0, "real": d}, "label": "Y"}
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

We define a, b, c, d coefficients as random numbers. Thus our Hamiltonian will be a random generated Hamiltonian which the smallest eigenvalue VQE algorithm will estimate.

In [3]:
a, b, c, d = (random(), random(), random(), random())
H = hamiltonian_operator(a, b, c, d)

Here we are using qiskit's ```ExactEigensolver``` class that will find the smallest eigenvalue of the given Hamiltonian via a classical algorithm. This value we will use as a reference for comparing it with VQE result.

In [4]:
exact_eigensolver = ExactEigensolver(H, k=2)
result_ee = exact_eigensolver.run()
reference_energy = result_ee['energy']
print('The exact ground state energy is: {}'.format(reference_energy))

The exact ground state energy is: -0.287693852578697


Now we need to create "good" ansatz states. First of all, note that every possible wavefunction $| \psi \rangle$can be presented as a vector: 
$$
| \psi \rangle = \begin{pmatrix}
c_1\\
e^{i \varphi} \cdot c_2
\end{pmatrix},
$$

where $c_1$ and $c_2$ are real numbers and $c_1^2 + c_2^2 = 1$, $\varphi$ is some phase ($\varphi \in (0, 2\pi)$). So, every wavefunction is described by 3 coordinates (values) ($c_1$, $c_2$ and $\varphi$) in 3D space. All possible states lie in one sphere in the 3D space. It's called the Bloch sphere and is presented in the following picture, that was taken from the book [3].

<img src="images/bloch_sphere.png" alt="drawing" width="300"/>

A "good" quantum state preparation circuit should be able to generate all possible states in the Bloch sphere [2]. Before quantum state preparation, our qubit is in $| 0 \rangle = \begin{pmatrix}
1\\
0
\end{pmatrix}$
state. That corresponds to the vertical position of the vector in the Bloch sphere. In order to generate any possible $| \psi \rangle$ we will use $R_x(\theta)$ and $R_y(\theta)$ gates. $R_x(\theta)$ corresponds to the rotation in the Bloch sphere around the *x* axis and $R_y(\theta)$ the rotation around the *y* axis. With these two rotations, one can have access to any point in the Bloch sphere. Here we show the matrix forms of $R_x(\theta)$ and $R_y(\theta)$ gates:

$$
R_x(\theta) = \begin{pmatrix}
cos(\frac{\theta}{2}) & -i \cdot sin(\frac{\theta}{2})\\
-i \cdot sin(\frac{\theta}{2}) & cos(\frac{\theta}{2})
\end{pmatrix}
\qquad
R_y(\theta) = \begin{pmatrix}
cos(\frac{\theta}{2}) & -sin(\frac{\theta}{2})\\
sin(\frac{\theta}{2}) & cos(\frac{\theta}{2})
\end{pmatrix}
$$

These two gates with there two parameters (two $\theta$s) will generate for us the trial (ansatz) wavefunctions. These two parameters will be in control of the Classical Computer and its optimization model.

The following method corresponds to the quantum state preparation circuit for given parameters (two $\theta$s).

In [5]:
def quantum_state_preparation(circuit, parameters):
    q = circuit.qregs[0] # q is the quantum register where the info about qubits is stored
    circuit.rx(parameters[0], q[0]) # q[0] is our one and only qubit XD
    circuit.ry(parameters[1], q[0])
    return circuit

Now we come to the part of the algorithm where we should do individual/separate measurements of expectation values of Hamiltonian Pauli terms. For one qubit case, we should think about only four possible Pauli terms: 4 basic Pauli operators $I$, $X$, $Y$, $Z$.

$$<H> = \langle \psi | H | \psi \rangle = a \cdot \langle \psi | I | \psi \rangle + b \cdot \langle \psi | Z | \psi \rangle + c \cdot \langle \psi | X | \psi \rangle + c \cdot \langle \psi | Y | \psi \rangle$$

For $I$ operator the  expectation value is always unity: $\langle \psi | I | \psi \rangle = \langle \psi | \psi \rangle = 1$. So, its contribution to the overall expectation value $<H>$ will be equal to $a \cdot \langle \psi | I | \psi \rangle = a$.

For rest of the Pauli operators, we should make the following remark: every quantum state $| \psi \rangle$ can be represented via different sets of basis vectors

$$| \psi \rangle = c_1^z \cdot | 0 \rangle + c_2^z \cdot | 1 \rangle = c_1^x \cdot | + \rangle + c_2^x \cdot | - \rangle = c_1^y \cdot | +i \rangle + c_2^y \cdot | -i \rangle,$$

where

\begin{align}
&\text{Z eigenvectors (basis gates)} \qquad
| 0 \rangle = \begin{pmatrix}
1\\
0
\end{pmatrix},
\qquad \qquad \; \; \;
| 1 \rangle = \begin{pmatrix}
0\\
1
\end{pmatrix},
\\
&\text{X eigenvectors (basis gates)} \qquad
| + \rangle = \frac{1}{\sqrt{2}} \begin{pmatrix}
1\\
1
\end{pmatrix},
\qquad \; \; \;
| - \rangle = \frac{1}{\sqrt{2}} \begin{pmatrix}
1\\
-1
\end{pmatrix},
\\
&\text{Y eigenvectors (basis gates)} \qquad
| +i \rangle = \frac{1}{\sqrt{2}} \begin{pmatrix}
1\\
i
\end{pmatrix},
\qquad 
| -i \rangle = \frac{1}{\sqrt{2}} \begin{pmatrix}
1\\
-i
\end{pmatrix}.
\end{align}

Here after each Pauli operator, we show its eigenvectors. The first presented eigenvectors for each Pauli has an eigenvalue equal to +1: $Z | 0 \rangle = +1| 0 \rangle$, $X | + \rangle = +1| + \rangle$, $Y | +i \rangle = +1| +i \rangle$. And the second presented eigenvectors for each Pauli has an eigenvalue equal to -1: $Z | 1 \rangle = -1| 1 \rangle$, $X | - \rangle = -1| - \rangle$, $Y | -i \rangle = -1| -i \rangle$. Now, let's calculate the expectation values of these Pauli operators: 

\begin{align}
\langle \psi | Z | \psi \rangle &= ({c_1^z}^* \cdot \langle 0 | + {c_2^z}^* \cdot \langle 1 |) Z (c_1^z \cdot | 0 \rangle + c_2^z \cdot | 1 \rangle) = {|c_1^z|}^2 - {|c_2^z|}^2
\\
\langle \psi | X | \psi \rangle &= ({c_1^x}^* \cdot \langle + | + {c_2^x}^* \cdot \langle - |) X (c_1^x \cdot | + \rangle + c_2^x \cdot | - \rangle) = {|c_1^x|}^2 - {|c_2^x|}^2
\\
\langle \psi | Y | \psi \rangle &= ({c_1^y}^* \cdot \langle +i | + {c_2^y}^* \cdot \langle -i |) Y (c_1^y \cdot | +i \rangle + c_2^y \cdot | -i \rangle) = {|c_1^y|}^2 - {|c_2^y|}^2
\end{align}

where we take into account that the inner product of orthonormal vectors is 0 (e.g. $\langle 0 | 1 \rangle = 0$, $\langle + | - \rangle = 0$, $\langle +i | -i \rangle = 0$). But what are these $|c|^2$s? The ${|c_1^z|}^2$ and ${|c_2^z|}^2$ are by definition the probabilities that after Z basis measurement (measuring is it $| 0 \rangle$ or is it $| 1 \rangle$) the quantum state $| \psi \rangle$ will become $| 0 \rangle$ or $| 1 \rangle$ respectively. In order to find that value, we should run our program with our trial $| \psi \rangle$  wavefunction and do $Z$ measurement on the qubit $N$ times (its named ```shots``` in code). The probability of finding the qubit after measurment in $| 0 \rangle$ state will be equal ${|c_1^z|}^2 = \frac{n_0}{N}$, where $n_0$ is the number of the $| 0 \rangle$ state measurments. Similarly, ${|c_2^z|}^2 = \frac{n_1}{N}$, where $n_1$ is the number of the $| 1 \rangle$ state measurments. Thus, the final expectation value will be equal to $\langle Z \rangle = \frac{n_0 - n_1}{N}$.

For $\langle X \rangle = \frac{n_+ - n_-}{N}$ and $\langle Y \rangle = \frac{n_{+i} - n_{-i}}{N}$ the expectation value estimation procedure stays the same. Here $n_+$ and $n_-$ are numbers of measurements in X basis that corresponds to outcomes $| + \rangle$ or $| - \rangle$ respectively. And $n_{+i}$ and $n_{-i}$ are numbers of measurements in Y basis that corresponds to outcomes $| +i \rangle$ or $| -i \rangle$ respectively. The difficulty comes from the fact that you may have the possibility to measure only in the $Z$ basis. To solve this difficulty we still do a $Z$ basis measurement, but, before that, we apply specific operators to our $| \psi \rangle$ state. We try to apply such an operator that after measuring the probability of $| 0 \rangle$ outcome will be equal to the probability of $| + \rangle$ ($| + \rangle$) outcome. And the probability of $| 1 \rangle$ outcome will be equal to the probability of measuring $| - \rangle$ ($| -i \rangle$) outcome. In mathematics it looks like this:

$$H_{gate} | \psi \rangle = H (c_1^x \cdot | + \rangle + c_2^x \cdot | - \rangle) = c_1^x \cdot | 0 \rangle + c_2^x \cdot | 1 \rangle,$$

where $H_{gate}$ is an operator such that $H_{gate} | + \rangle = | 0 \rangle$ and $H_{gate} | - \rangle = | 1 \rangle$.

$$Y_{gate} | \psi \rangle = Y_{gate} (c_1^y \cdot | +i \rangle + c_2^y \cdot | -i \rangle) = c_1^y \cdot | 0 \rangle + c_2^y \cdot | 1 \rangle,$$

where $Y_{gate}$ is an operator such that $Y_{gate} | +i \rangle = | 0 \rangle$ and $Y_{gate} | -i \rangle = | 1 \rangle$.

This kind of operators can be easily found:

$$
H_{gate} = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1\\
1 & -1
\end{pmatrix}
\qquad
Y_{gate} = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & -i\\
1 & i
\end{pmatrix}
$$
BTW the $H_{gate}$ is the well known as Hadamard gate (common notation of it is $H$, but we don't use this notation because we already have used the $H$ letter for the Hamiltonian operator). In the following code you can see an implementation of these gates via qiskit's ```u2``` gate:

$$
u2(\varphi, \lambda) = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & -e^{i \lambda}\\
e^{i \varphi} & e^{i (\varphi + \lambda)}
\end{pmatrix}.
$$

In [6]:
H_gate = U2Gate(0, np.pi).to_matrix()
print("H_gate:")
print((H_gate * np.sqrt(2)).round(5))

Y_gate = U2Gate(0, np.pi/2).to_matrix()
print("Y_gate:")
print((Y_gate * np.sqrt(2)).round(5))

H_gate:
[[ 1.+0.j  1.-0.j]
 [ 1.+0.j -1.+0.j]]
Y_gate:
[[ 1.+0.j -0.-1.j]
 [ 1.+0.j  0.+1.j]]


Finally, we are ready to go! In this code, we define a method that will create circuits with its specific measurement ($Z$ or $X$ or $Y$ measurments).

In [7]:
def vqe_circuit(parameters, measure):
    """
    Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'Z' stands for Z measurement.
    :return: quantum circuit.
    """
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    circuit = QuantumCircuit(q, c)

    # quantum state preparation
    circuit = quantum_state_preparation(circuit, parameters)

    # measurement
    if measure == 'Z':
        circuit.measure(q[0], c[0])
    elif measure == 'X':
        circuit.u2(0, np.pi, q[0])
        circuit.measure(q[0], c[0])
    elif measure == 'Y':
        circuit.u2(0, np.pi/2, q[0])
        circuit.measure(q[0], c[0])
    else:
        raise ValueError('Not valid input for measurement: input should be "X" or "Y" or "Z"')

    return circuit

We are going to work with python dictionaries and the following method will help us to avoid some errors. It will return the value (number of $| 0 \rangle$ state measurements) of a corresponding key (e.g ```'0'```), if it exists, else return $0$ (never measured ```'0'``` state).

In [8]:
def get_or_else_zero(d: dict, key: str):
    """
    Utility for working with dictionaries. If key is missing
    than return 0 otherwise the corresponding value.
    :param dict: the dictionary.
    :param key: key (string) in interest.
    :return: 0 or value of corresponding key.
    """
    value = 0
    if key in d:
        value = d[key]
    return value

Here is the quantum module part. The ```quantum_module``` finds the expectation values of a Pauli operator.

In [9]:
def quantum_module(parameters, measure):
    # measure
    if measure == 'I':
        return 1
    elif measure == 'Z':
        circuit = vqe_circuit(parameters, 'Z')
    elif measure == 'X':
        circuit = vqe_circuit(parameters, 'X')
    elif measure == 'Y':
        circuit = vqe_circuit(parameters, 'Y')
    else:
        raise ValueError('Not valid input for measurement: input should be "I" or "X" or "Z" or "Y"')
    
    shots = 1000
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    expectation_value = (get_or_else_zero(counts, '0') - get_or_else_zero(counts,'1')) / shots
    
    return expectation_value

The ```pauli_operator_to_dict``` creates a  dictionary from the ```WeightedPauliOperator``` object (our Hamiltonian is encoded in this object). 

In [10]:
def pauli_operator_to_dict(pauli_operator):
    """
    from WeightedPauliOperator return a dict:
    {I: 0.7, X: 0.6, Z: 0.1, Y: 0.5}.
    :param palui_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}

    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff

    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

The following is the main method that takes parameters for ansatz state preparation and returns the corresponding expectation value of our Hamiltonian. For each Pauli term, we create separate quantum modules that are calculating the expectation value of each Pauli. Then, all expectation values of Pauli operators multiplied by there corresponding coefficients ($a$, $b$, $c$, $d$) are summed.

In [11]:
def vqe(parameters):
        
    # quantum_modules
    quantum_module_I = get_or_else_zero(pauli_dict, 'I') * quantum_module(parameters, 'I')
    quantum_module_Z = get_or_else_zero(pauli_dict, 'Z') * quantum_module(parameters, 'Z')
    quantum_module_X = get_or_else_zero(pauli_dict, 'X') * quantum_module(parameters, 'X')
    quantum_module_Y = get_or_else_zero(pauli_dict, 'Y') * quantum_module(parameters, 'Y')
    
    # summing the measurement results
    classical_adder = quantum_module_I + quantum_module_Z + quantum_module_X + quantum_module_Y
    
    return classical_adder

This is the final part of the code where we are minimizing (optimizing) the returned value from the ```vqe``` method by changing parameters for the quantum state preparation circuit (trial/ansatz $| \psi \rangle$).

In [12]:
parameters_array = array([np.pi, np.pi])
tol = 1e-3 # tolerance for optimization precision.

vqe_result = minimize(vqe, parameters_array, method="Powell", tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))

The exact ground state energy is: -0.287693852578697
The estimated ground state energy from VQE algorithm is: -0.288761574814124


This is it! VQE algorithm for one qubit. But what should we do if we have multiple qubits and we should estimate not the expectation value of a single Pauli operator, but the expectation value of a product of Pauli operators? For example, the Hamiltonian could look like this:

$$<H> = \langle \psi | H | \psi \rangle = 0.4 \cdot \langle \psi | IX | \psi \rangle + 0.6 \cdot \langle \psi | IZ | \psi \rangle + 0.8 \cdot \langle \psi | XY | \psi \rangle$$

And the algorithm should calculate the expectation value of each Pauli product. The most not trivial case is the $XY$ Pauli term and we will consider only this case. Note that for tensor products of operators like $XY$ Pauli term (actually $XY = X \otimes Y$):

$$U_1 \otimes U_2 | \psi_1 \rangle \otimes | \psi_2 \rangle =  U_1 | \psi_1 \rangle \otimes U_2 | \psi_2 \rangle,$$

were $U_1$ and $U_2$ are some unitary operators acting on two qubits with separate $| \psi_1 \rangle$ and $| \psi_2 \rangle$ wavefunctions. This way it can be shown that tensor products of the eigenvectors of $X$ and $Y$ are the eigenvectors for $XY$ Pauli term. This is true for any Pauli term. Also, notice that all eigenvectors of  Pauli product terms have eigenvalues equal to +1 or -1. Thus, the eigenvectors for $XY$ that have +1 eigenvalue are $| + \rangle \otimes | +i \rangle = | ++i \rangle$, $| - \rangle \otimes | -i \rangle = | --i \rangle$, and the eigenvectors that have -1 eigenvalue are $| + \rangle \otimes |-+i \rangle = | +-i \rangle$, $| + \rangle \otimes | -i \rangle = | +-i \rangle$. As an example:

$$XY | + -i \rangle= X \otimes Y | + \rangle \otimes | -i \rangle =  X | + \rangle \otimes Y | -i \rangle = 
-| + \rangle \otimes | -i \rangle = - | + -i \rangle$$

The measurement logic stays the same. So, for $\langle XY \rangle$ one should apply $H_{gate}$ on the first qubit and $Y_{gate}$ on the second qubit (or like in tensor product notation $H_{gate} \otimes Y_{gate}$) before Z basis measurement. To see that we should represent the combined quantum state $| \psi \rangle = | \psi_1 \rangle \otimes | \psi_2 \rangle$ in the basis of the eigenvectors of $XY$ operator:

$$H_{gate} \otimes Y_{gate} | \psi \rangle = H_{gate} \otimes Y_{gate} (c_1^{XY}| ++i \rangle + c_2^{XY}| +-i \rangle + c_3^{XY}| -+i \rangle + c_4^{XY}| --i \rangle) = c_1^{XY}| 00 \rangle + c_2^{XY}| 01 \rangle + c_3^{XY}| 10 \rangle + c_4^{XY}| 11 \rangle,$$

where $|c|^2$ are probabilities of measuring in the corresponding eigenvectors of $XY$. By doing similar calculations as we did for single Paulis, it can be shown that the expectation $\langle \psi | XY | \psi \rangle$ is equal to the sum of $|c|^2$s with -1 coefficient for those eigenvectors which eigenvalue is -1 and with +1 coefficient for those eigenvectors which eigenvalue is -1.

This way one can scale this solution for any Hamiltonian.

[1] [A. Peruzzo et al., Nature Communications, "A variational eigenvalue solver on a photonic quantum processor" (2014).](https://www.nature.com/articles/ncomms5213?origin=ppub)

[2] [Michał Stęchły, "Variational Quantum Eigensolver explained"](https://www.mustythoughts.com/post/variational-quantum-eigensolver-explained).

[3] [M.A. Nielsen, I.L. Chuang, Cambridge University Press New York, "Quantum Computation and Quantum Information: 10th Anniversary Edition
10th" (2011)](https://www.cambridge.org/am/academic/subjects/physics/quantum-physics-quantum-information-and-quantum-computation/quantum-computation-and-quantum-information-10th-anniversary-edition?format=HB)